In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 59.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 105.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 93.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 38.4 MB/s eta 0:00:00


# Evaluating the Deletion model

In [3]:
import pandas as pd
import sys
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import tqdm
import torch

sys.path.append(r'/content/drive/MyDrive/University/PML/A1')  # Add the path to the models directory

class DeletionModel:
    def __init__(self, bad_words_path):
        self.bad_words = pd.read_csv(bad_words_path, header=None)[0].tolist()

    def train(self):
        print("No need to train this model")

    def detoxify(self, X_test):
        result = []
        for sentence in X_test:
            result_sentence = []
            for word in sentence.split():
                if word not in self.bad_words:
                    result_sentence.append(word)
            result.append(" ".join(result_sentence))
        return result

class StyleTransferAccuracy:
    def __init__(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.tokenizer = RobertaTokenizer.from_pretrained('SkolkovoInstitute/roberta_toxicity_classifier')
        self.model = RobertaForSequenceClassification.from_pretrained('SkolkovoInstitute/roberta_toxicity_classifier')

        # Move the model to the GPU
        self.model = self.model.to(self.device)

    def classify_preds(self, batch_size, preds):
        print('Calculating style of predictions')
        results = []

        for i in tqdm.tqdm(range(0, len(preds), batch_size)):
            batch = self.tokenizer(preds[i:i + batch_size], return_tensors='pt', padding=True)

            # Move the batch to the GPU
            batch = {key: value.to(self.device) for key, value in batch.items()}

            result = self.model(**batch)['logits'].argmax(1).float().data.tolist()
            results.extend([1 - item for item in result])

        return results


In [4]:
test_data = pd.read_csv(r'/content/drive/MyDrive/University/PML/A1/Data/testing.csv')

In [5]:
bad_words_path = r"/content/drive/MyDrive/University/PML/A1/Data/bad-words.csv"
model = DeletionModel(bad_words_path)
x = test_data['toxic'].tolist()

output = model.detoxify(x)
output

KeyboardInterrupt: ignored

In [ ]:
# from metric.style_trasnfer_accuracy import StyleTransferAccuracy
import numpy as np
sta = StyleTransferAccuracy()

accuracies = sta.classify_preds(64, output)

np.mean(accuracies)

### Calculating the average style transfer accuracy for the deletion model

In [ ]:
import numpy as np

np.mean(accuracies)

## Calculating cos similarity

In [ ]:
output[1]

'Who the has been going through my stuff?!'

In [ ]:
len(x)

115556

In [ ]:
x_reduced = x[:10000]
output_reduced = output[:10000]

In [ ]:
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased').to(device)

def batch_embeddings(sentences):
    inputs = tokenizer(sentences, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
    outputs = model(**inputs)

    # Take the average of the hidden states to get sentence embeddings
    embeddings = torch.mean(outputs.last_hidden_state, dim=1).detach().cpu().numpy()
    return embeddings

def calculate_cosine_similarity(list1, list2, batch_size=32):
    total_similarities = []
    for i in range(0, len(list1), batch_size):
        # Process each batch
        batch1 = list1[i:i+batch_size]
        batch2 = list2[i:i+batch_size]

        embeddings1 = batch_embeddings(batch1)
        embeddings2 = batch_embeddings(batch2)

        # Calculate cosine similarity for the batch
        similarities = cosine_similarity(embeddings1, embeddings2).diagonal()
        total_similarities.extend(similarities)
    return total_similarities

cosine_similarities = calculate_cosine_similarity(x_reduced, output_reduced, batch_size=64)

# Calculate the mean cosine similarity
mean_similarity = np.mean(cosine_similarities)
print(f"Mean Cosine Similarity: {mean_similarity}")


Mean Cosine Similarity: 0.981300950050354


# Evaluating the T5 model

In [3]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Define constants and model path
MODEL_PATH = r"../models/final_solution"
MAX_LENGTH = 128

# Load the trained model and tokenizer from the specified model path
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_PATH)

# Determine the device (GPU or CPU) to use for inference
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the determined device
model.to(device)



T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [19]:
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd

# Define the input prompt to be detoxified
x = pd.read_csv(r'../data/interim/testing.csv')['toxic'].tolist()

# Tokenize the input prompts
tokenized_inputs = tokenizer(
    x,
    return_tensors='pt',
    truncation=True,
    max_length=MAX_LENGTH,
    padding=True
)

# Create a TensorDataset and DataLoader for batch processing
dataset = TensorDataset(tokenized_inputs['input_ids'], tokenized_inputs['attention_mask'])
dataloader = DataLoader(dataset, batch_size=64)

# List to store the detoxified texts
detoxified_texts = []

# Perform inference in batches
for batch in dataloader:
    # Move tensors to the same device as the model
    batch = tuple(t.to(device) for t in batch)
    inputs = {'input_ids': batch[0], 'attention_mask': batch[1]}

    with torch.no_grad():
        # Generate the detoxified output sequence for the batch
        outputs = model.generate(
            **inputs,
            max_length=MAX_LENGTH,
            num_return_sequences=1
        )

    # Decode the generated output tensors to get the detoxified text
    batch_detoxified_texts = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
    detoxified_texts.extend(batch_detoxified_texts)

# Now detoxified_texts contains all the detoxified outputs
# Print the first few detoxified texts as a sample
for text in detoxified_texts[:5]:
    print(f"Detoxified text: {text}")


Detoxified text: call out your butchers and I'll tell you.
Detoxified text: Who the fuck has been going through my stuff?!
Detoxified text: he can still die.
Detoxified text: Yeah, that's the fucker's name.
Detoxified text: I would take you on my shoulders, like, I'd strap you up, and I'd be like, let's go through hell.


In [20]:
import numpy as np
import sys
sys.path.append("../src/")
from metric.style_trasnfer_accuracy import StyleTransferAccuracy
# Clears the GPU cache
torch.cuda.empty_cache()

sta = StyleTransferAccuracy()

accuracies = sta.classify_preds(1, detoxified_texts)

np.mean(accuracies)

Some weights of the model checkpoint at SkolkovoInstitute/roberta_toxicity_classifier were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Calculating style of predictions


100%|██████████| 10000/10000 [03:09<00:00, 52.77it/s]


0.1762

In [21]:
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased').to(device)

def batch_embeddings(sentences):
    inputs = tokenizer(sentences, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
    outputs = model(**inputs)

    # Take the average of the hidden states to get sentence embeddings
    embeddings = torch.mean(outputs.last_hidden_state, dim=1).detach().cpu().numpy()
    return embeddings

def calculate_cosine_similarity(list1, list2, batch_size=32):
    total_similarities = []
    for i in range(0, len(list1), batch_size):
        # Process each batch
        batch1 = list1[i:i+batch_size]
        batch2 = list2[i:i+batch_size]

        embeddings1 = batch_embeddings(batch1)
        embeddings2 = batch_embeddings(batch2)

        # Calculate cosine similarity for the batch
        similarities = cosine_similarity(embeddings1, embeddings2).diagonal()
        total_similarities.extend(similarities)
    return total_similarities

cosine_similarities = calculate_cosine_similarity(x, detoxified_texts, batch_size=64)

# Calculate the mean cosine similarity
mean_similarity = np.mean(cosine_similarities)
print(f"Mean Cosine Similarity: {mean_similarity}")


d:\Program Files\miniconda\envs\PMLA1\lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Electronica\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


OutOfMemoryError: CUDA out of memory. Tried to allocate 54.00 MiB. GPU 0 has a total capacty of 4.00 GiB of which 0 bytes is free. Of the allocated memory 3.27 GiB is allocated by PyTorch, and 180.53 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF